In [3]:
df = spark.read.table("Silver_SessionData")

from pyspark.sql.functions import to_date, hour

df = df.withColumn("Date", to_date("Time_stamp"))
df = df.withColumn("Hour", hour("Time_stamp"))

StatementMeta(, 74bdeb95-e958-4303-bac4-3f48668f8250, 3, Finished, Available, Finished)

In [3]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import functions as F

def traffic_trends(df):
    # Performed 1 aggregation grouped on columns: 'URI', 'Tenant'
    df = df.groupBy('URI', 'Tenant').agg(F.count('URI').alias('URI_count'))
    df = df.dropna()
    df = df.sort(df['URI'].asc(), df['Tenant'].asc())
    return df

df_traffic_trends = traffic_trends(df)
df_traffic_trends.write.mode("overwrite").saveAsTable("Gold_Traffic_trends")

StatementMeta(, 3ea42fdc-8bfd-4d9c-b8da-423a5e3dc0a6, 28, Finished, Available, Finished)

In [5]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import functions as F

def Navigation_path(df):
    # Remove rows where Referrer is null, empty, or NaN
    df = df.filter(
        (F.col('Referrer').isNotNull()) &
        (F.col('Referrer') != '') &
        (~F.isnan(F.col('Referrer')))
    )

    # Perform aggregation
    df = df.groupBy('Tenant', 'URI', 'Referrer').agg(F.count('IP_Address').alias('IP_Address_count'))

    # Sort the DataFrame
    df = df.sort(df['Tenant'].asc(), df['URI'].asc(), df['Referrer'].asc())
    
    return df

df_Nav_Path = Navigation_path(df)
df_Nav_Path.write.mode("overwrite").saveAsTable("Gold_Navigation_Path")



StatementMeta(, 3ea42fdc-8bfd-4d9c-b8da-423a5e3dc0a6, 30, Finished, Available, Finished)

In [7]:
import pandas as pd
from pyspark.sql import functions as F

def Session_time(df):
    # Sort by columns: 'Tenant', 'Session_ID', 'Time_stamp'
    df = df.sort(df['Tenant'].asc(), df['Session_ID'].asc(), df['Time_stamp'].asc())
    # Group by 'Tenant' and 'Session_ID' and get first and last timestamps
    df = df.groupBy('Tenant', 'Session_ID').agg(
        F.last('Time_stamp').alias('Time_stamp_last'),
        F.first('Time_stamp').alias('Time_stamp_first')
    )
    df = df.dropna()
    pandas_df = df.toPandas()
    pandas_df['Time_stamp_last'] = pd.to_datetime(pandas_df['Time_stamp_last'])
    pandas_df['Time_stamp_first'] = pd.to_datetime(pandas_df['Time_stamp_first'])
    pandas_df['Session_Time'] = (pandas_df['Time_stamp_last'] - pandas_df['Time_stamp_first']).dt.total_seconds()

    # Filter out sessions longer than 30 minutes (1800 seconds)
    pandas_df = pandas_df[pandas_df['Session_Time'] <= 1800]
    return pandas_df

df_Session_time = Session_time(df)
spark_df_Session_time = spark.createDataFrame(df_Session_time)

# Overwrite the existing table with updated schema and data
spark_df_Session_time.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("Gold_AvgSession_time")

StatementMeta(, 405a93c5-504e-4911-a6f1-57254e44c8a2, 9, Finished, Available, Finished)

In [7]:
from pyspark.sql import functions as F

def Peak_hours(df):
    df = df.dropna()
    # Convert integer hour to time string and rename column
    df = df.withColumn('Formatted_Hour', F.date_format(F.to_timestamp(df['Hour'].cast('string'), 'H'), 'h:mm a'))
    # Drop original Hour column
    df = df.drop('Hour')
    # Rename formatted column to Hour
    df = df.withColumnRenamed('Formatted_Hour', 'Hour')
    # Sort by Hour and Tenant
    df = df.sort(df['Hour'].asc(), df['Tenant'].asc())
    return df

df_Peak_hours = Peak_hours(df)
df_Peak_hours.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("Gold_Peak_hours")

StatementMeta(, 74bdeb95-e958-4303-bac4-3f48668f8250, 29, Finished, Available, Finished)